# Projeto

**Objetivo**: Desenvolver um CRUD utilizando um arquivo JSON para "persistir" os dados e aplicar algumas outras funcionalidades que seriam interessantes no sistema.

**Requisitos**: O projeto deve conter as seguintes funcionalidades/ características:

- Utilizar JSON para persistir os dados
- Função de criar o arquivo JSON (amanda) OK
- Função para adicionar registros (através do *input*, ou de um conjunto de dados históricos, por exemplo) (ricardo)
- Função para realizar o *udpate* (atualização) das informações (leticia)
- Função para excluir registros (adriely)
- Função para realizar a leitura de registros (printar os dados na tela do consumidor x, lista) (leticia)
- Função para filtrar informações dos dados (Amanda) +/- OK
- Função para aplicar alguma regra de negócio (varia de acordo com o sistema, exemplo, grupo criou um sistema de consulta de colaboradores, logo deseja-se ter uma função que calcula o faturamento de todos os funcionários que são vendedores) (lorrany)
- Função que retorne alguma estatística dos dados (lorrany)
- menu (daniel)
- Exemplos de funcionamento de cada uma das funções acima

**Pontos importantes**:

- Organização de código no ``jupyter notebook`` (não precisa de ppt, no dia apenas o notebook será necessário)
- Definição sobre o que será o sistema (livre e definido pelo grupo)
- Apresentação do sistema e funcionalidades (assim como os exemplos). Não precisa ser o grupo todo apresentando
- Gerenciamento de erros, como no caso de num input o usuário colocar 'abc' num campo de valor inteiro
- A nota do grupo será a nota de cada integrante
- Tempo de apresentação: de 15 a 20 minutos
- A princípio todos integrantes devem entregar o mesmo notebook no sistema do LMS, para fins de retorno de nota
- Não precisa se ater somente aos 10 requisitos acima, pode ir além :)

banco de dados
https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset

## Função para a leitura de dados de um csv ou json e transformação em uma lista de dicionários

In [191]:
import json
import csv
def importar_dados(extensao:str):
    dados = list()
    
    opcoes = arquivos_disponiveis(extensao)
    respostas_validas = list(opcoes.keys())
    pergunta = 'Informe o numero do arquivo que deseja importar'
    str_arquivo = opcoes[validacao_respostas(pergunta, respostas_validas)]
    
    if extensao == '.json':
        with open(str_arquivo, "r", encoding = "utf-8") as f:
            dados = json.load(f)

    else:
        with open(str_arquivo, "r") as arquivo:
            arquivo_csv = csv.reader(arquivo, delimiter=",")
            next(arquivo_csv)  # pula a primeira linha (cabeçalho)
            for linha in enumerate(arquivo_csv):
                #print(linha)
                Customer_ID, Age, Gender, Item_Purchased,Category, Purchase_Amount, Location, Size, Color, Season, Review_Rating, Subscription_Status, Payment_Method, Shipping_Type, Discount_Applied, Promo_Code_Used, Previous_Purchases, Preferred_Payment_Method, Frequency_of_Purchases = linha[1]
                #print('ID', Customer_ID,'Idade', Age,'Genero', Gender,'Item_Comprado',Item_Purchased,
                #                          'Categoria', Category,'Metodo_Pagamento', Payment_Method)
                # 
                dados.append({'ID': int(Customer_ID), #numérica
                                    'Idade': int(Age), #numérica
                                    'Genero': Gender, #categórica
                                    'Item_Comprado':Item_Purchased, #categórica
                                    'Categoria': Category, #categórica
                                    'Preco': float(Purchase_Amount), #numérica
                                    'Metodo_Pagamento': Payment_Method, #categórica
                                    'Quantidade_Compras_Anteriores': int(Previous_Purchases), #numérica
                                    'Frequencia_Compra': Frequency_of_Purchases #categórica
                                    })
    return dados
    

In [195]:
dados = importar_dados('.csv')
dados_traduzidos = traduzindo_lista_dict(dados)

Lista de arquivos .CSV disponíveis
1	shopping_trends.csv


# Traduzir os valores da variável ``lista_vendas()``

In [124]:
def traduzindo_lista_dict(lista: list):
    for chave in list(lista[0].keys()):
        if type(lista_vendas[0][chave]) == str:
            categorias = lista_de_valores_sem_repeticoes(lista, chave)
            dict_traducao = traduz_palavras(categorias)
            for i in range(len(lista)):
                lista[i][chave] = dict_traducao[lista[i][chave]]
    return lista

## Função que retorna uma lista de valores sem repetição

In [67]:
def lista_de_valores_sem_repeticoes(lista_de_dicionarios, chave):
  """
  Recebe uma lista de dicionários.
  A função retorna lista dos valores encontrados para a chave, sem repetições de valores.

  Args:
    lista_de_dicionarios: Uma lista de dicionários.
    chave: A chave a ser pesquisada.

  Returns:
    Uma lista.
  """
  lista_de_valores = set()
  for dicionario in lista_de_dicionarios:
    if chave in dicionario:
      lista_de_valores.add(dicionario[chave])
  return list(lista_de_valores)

## Função para traduzir palavras

In [130]:

from translate import Translator
def traduz_palavras(lista_de_palavras):
  """
  Recebe uma lista de palavras em inglês e retorna um dicionário onde 
  a chave é a palavra da lista e o valor é a tradução em português.

  Args:
    lista_de_palavras: Uma lista de palavras em inglês.

  Returns:
    Um dicionário.
  """
  dicionario_de_traducao = {}
  for palavra in lista_de_palavras:
    traducao = Translator(to_lang='pt-br', from_lang='english')
    dicionario_de_traducao[palavra] = traducao.translate(palavra)
  return dicionario_de_traducao


## Exportando os dados traduzidos

In [132]:
lista_vendas = leitura_csv()
lista_vendas_trad = traduzindo_lista_dict(lista_vendas)

In [139]:
exportar_json(lista_vendas_trad)

Arquivo exportado com sucesso!


# Função que lista os arquivos disponíveis

In [172]:
import os
def arquivos_disponiveis(extensao: str):
    pergunta = 'Informe o numero do arquivo que deseja importar'
    opcoes = dict()
    arquivos = os.listdir() # lista todos os arquivos presentes na pasta
    arquivos_extensao = [arquivo for arquivo in arquivos if arquivo.endswith(extensao)] # filtra pela extensao
    print(f'Lista de arquivos {extensao.upper()} disponíveis')
    for i in range(len(arquivos_extensao)):
        print(f'{i+1}\t{arquivos_extensao[i]}')
        opcoes.update({str(i+1):arquivos_extensao[i]})
    return opcoes
        

## Função de filtro

In [209]:
def filtro(lista: list, categoria:str, valor_categoria, tipo):
    if tipo == 'Categorico':
        filtro = [ i for i in lista if i[categoria] == valor_categoria]
    if tipo == 'Maior que':
        filtro = [ i for i in lista if i[categoria] > valor_categoria]
    if tipo == 'Maior e iqual a':
        filtro = [ i for i in lista if i[categoria] >= valor_categoria]
    if tipo == 'Menor que':
        filtro = [ i for i in lista if i[categoria] < valor_categoria]
    if tipo == 'Menor e iqual a':
        filtro = [ i for i in lista if i[categoria] <= valor_categoria]
    if tipo == 'Igual a':
        filtro = [ i for i in lista if i[categoria] == valor_categoria]
    return filtro
    
    
dados_filtrados = filtro(dados_traduzidos, 'Idade',30,'Maior e iqual a')

## Função para validação de respostas do usuário

In [30]:
def validacao_respostas(pergunta, respostas_validas):
  """
  Validação de respostas. A função valida respostas de um caracter
  É passado como parâmetro:
    - A pergunta que se quer fazer (str)
    - Uma lista com as respostas válidas

  Caso o usuário digite algo diferente das respostas válidas,
  a função repetirá a pergunta até que o usuário responda um valor válido.
  E só retornará opções válidas de valores
  """
  resp = input(pergunta).upper()[0]# apenas 1ªletra
  pergunta = 'Resposta inválida! ' + pergunta
  while not resp in respostas_validas:
    
    resp = input(pergunta).upper()[0]# apenas 1ªletra
    #resp = input(pergunta).upper()[0]# apenas 1ªletra
  return resp

## Função para exportar uma lista de dicionários para um arquivo .json

In [138]:
import json
import os.path
def exportar_json(lista_vendas: list):
    """
    A função exporta a lista_vendas para um arquivo.
    O arquivo será nomeado pelo usuário
        Caso o arquivo já exista na pasta, o usuário será questionado se quer sobrescrever.
    O arquivo será no formato .json
    Será importado na mesma pasta que se encontra o script
    Args:
        lista_vendas (list): lista de dicionários com as informações de venda
    """
    nome_arquivo = input("Informe o nome do arquivo a ser criado:\t").lower()
    

    # retirando os possíveis espaços do nome
    qtd_espaco = nome_arquivo.count(' ')
    if qtd_espaco > 0:
        nome_arquivo =  nome_arquivo.replace(" ", "_", qtd_espaco)
    nome_arquivo += '.json'
    
    if os.path.exists(nome_arquivo) == True:
        pergunta = 'Já existe um arquivo com esse nome.\nDeseja sobrescrever?[S]  Sim\t [N] Não\t'
        resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
        if resp_sobrescrever == 'N':
            print('O arquivo não foi modificado.')
            return
    try:
        json_vendas = json.dumps(lista_vendas, indent=2,ensure_ascii=False)
        arquivo = open(nome_arquivo, 'w', encoding = "utf-8") 
        arquivo.write(json_vendas) 
        arquivo.close()
        print("Arquivo exportado com sucesso!")
    except:
        print("Erro ao exportar arquivo")
    return